In [1]:
import sklearn
import pandas as pd
import numpy as np
from gensim.models import KeyedVectors

from tensorflow import keras
from tensorflow.keras.preprocessing import text, sequence
from tensorflow.keras.regularizers import l2

In [2]:
COMMENT = 'comment_text'
LABELS = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [3]:
base_path = 'jigsaw-toxic-comment-classification-challenge'
train = pd.read_csv(f'{base_path}/train.csv')
test = pd.read_csv(f'{base_path}/test.csv')
submission = pd.read_csv(f'{base_path}/sample_submission.csv')
test_labels = pd.read_csv(f'{base_path}/test_labels.csv')

In [4]:
EMBEDDING_FILE = f'{base_path}/GoogleNews-vectors-negative300.bin'

wv = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)
embeddings_index = {w:wv.get_vector(w) for w in wv.index2word}

all_embs = np.stack(embeddings_index.values())
emb_mean = all_embs.mean()
emb_std = all_embs.std()
emb_mean,emb_std

/home/shanger_lin/.pyenv/versions/3.6.4/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2850: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  if self.run_code(code, result):


(-0.003527845, 0.13315111)

# Problem understanding

# Dataset generation

# Metrics define

# Validation strategy

# Data processing (extract useful information)

In [5]:
MAX_TOKENS = 30000
MAX_SENTENSE_LEN = 100
EMBEDDING_SIZE = all_embs.shape[1]

In [6]:
x_train = train["comment_text"].values
y_train = train[LABELS].values
x_test = test["comment_text"].values

tokenizer = text.Tokenizer(num_words=MAX_TOKENS)
tokenizer.fit_on_texts(list(x_train) + list(x_test)) # is it a proper trick?

x_train, x_test = map(tokenizer.texts_to_sequences, [x_train, x_test])
x_train, x_test = map(lambda x: sequence.pad_sequences(x, maxlen=MAX_SENTENSE_LEN), [x_train, x_test])


In [7]:
word_index = tokenizer.word_index
nb_words = min(MAX_TOKENS, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, EMBEDDING_SIZE))
for word, i in word_index.items():
    if i >= MAX_TOKENS: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: 
        embedding_matrix[i] = embedding_vector

# Data understanding & visualization

# De-noise (no drop data)

# Feature engineering

# Offline augmentation

# Standarization

# Scaling

# Normalization

# Feature selection

# Data selection

# Optimization

In [8]:
model_input = keras.Input(shape=(MAX_SENTENSE_LEN, ))
x = keras.layers.Embedding(MAX_TOKENS, EMBEDDING_SIZE, mask_zero=True, weights=[embedding_matrix], embeddings_regularizer=l2(1e-5))(model_input)
x = keras.layers.GRU(64, return_sequences=True, activation="relu", kernel_regularizer=l2(1e-5))(x)
x = keras.layers.SpatialDropout1D(0.2)(x)
x = keras.layers.GRU(32, return_sequences=False, activation="relu", kernel_regularizer=l2(1e-5))(x)
x = keras.layers.Dropout(0.2)(x)
x = keras.layers.Dense(32, activation="relu")(x)
model_output = keras.layers.Dense(len(LABELS), activation="sigmoid")(x)

model = keras.Model(inputs=model_input, outputs=model_output)
model.compile(keras.optimizers.Adam(3e-4), loss='binary_crossentropy', metrics=['acc'])

hist = model.fit(x_train, 
                 y_train, 
                 batch_size=256,
                 shuffle=True,
                 epochs=100, 
                 validation_split=0.05,
                 callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss',
                                                          patience=1, 
                                                          verbose=1)], 
                 verbose=1)

y_pred = model.predict(x_test)
submission[LABELS] = y_pred
submission.to_csv('91.submission.csv', index=False)

# ~ 0.95921

Epoch 1/100


/home/shanger_lin/.pyenv/versions/3.6.4/lib/python3.6/site-packages/tensorflow/python/framework/indexed_slices.py:432: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


593/593 [==============================] - 260s 438ms/step - loss: 1.6018 - acc: 0.6797 - val_loss: 0.7545 - val_acc: 0.9952
Epoch 2/100
593/593 [==============================] - 261s 439ms/step - loss: 0.4564 - acc: 0.9587 - val_loss: 0.2636 - val_acc: 0.9952
Epoch 3/100
593/593 [==============================] - 258s 435ms/step - loss: 0.1807 - acc: 0.9764 - val_loss: 0.1281 - val_acc: 0.9952
Epoch 4/100
593/593 [==============================] - 257s 433ms/step - loss: 0.1012 - acc: 0.9808 - val_loss: 0.0884 - val_acc: 0.9952
Epoch 5/100
593/593 [==============================] - 256s 432ms/step - loss: 0.0753 - acc: 0.9846 - val_loss: 0.0739 - val_acc: 0.9952
Epoch 6/100
593/593 [==============================] - 257s 433ms/step - loss: 0.0650 - acc: 0.9859 - val_loss: 0.0685 - val_acc: 0.9952
Epoch 7/100
593/593 [==============================] - 256s 431ms/step - loss: 0.0596 - acc: 0.9883 - val_loss: 0.0664 - val_acc: 0.9952
Epoch 8/100
593/593 [==============================] 

In [9]:
model_input = keras.Input(shape=(MAX_SENTENSE_LEN, ))
x = keras.layers.Embedding(MAX_TOKENS, EMBEDDING_SIZE, mask_zero=True, weights=[embedding_matrix], embeddings_regularizer=l2(1e-5))(model_input)
x = keras.layers.Bidirectional(keras.layers.GRU(64, return_sequences=True, activation="relu", recurrent_dropout=0.1))(x)
x = keras.layers.SpatialDropout1D(0.2)(x)
x = keras.layers.Bidirectional(keras.layers.GRU(32, return_sequences=True, activation="relu", recurrent_dropout=0.1))(x)
x = keras.layers.GlobalMaxPool1D()(x)
x = keras.layers.Dense(32, activation="relu")(x)
model_output = keras.layers.Dense(len(LABELS), activation="sigmoid")(x)

model = keras.Model(inputs=model_input, outputs=model_output)
model.compile(keras.optimizers.Adam(3e-4), loss='binary_crossentropy', metrics=['acc'])

hist = model.fit(x_train, 
                 y_train, 
                 batch_size=256,
                 shuffle=True,
                 epochs=100, 
                 validation_split=0.05,
                 callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss',
                                                          patience=1, 
                                                          verbose=1)], 
                 verbose=1)

y_pred = model.predict(x_test)
submission[LABELS] = y_pred
submission.to_csv('92.submission.csv', index=False)

# ~ 0.96804

Epoch 1/100


/home/shanger_lin/.pyenv/versions/3.6.4/lib/python3.6/site-packages/tensorflow/python/framework/indexed_slices.py:432: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


593/593 [==============================] - 418s 706ms/step - loss: 1.5571 - acc: 0.9434 - val_loss: 0.7238 - val_acc: 0.9950
Epoch 2/100
593/593 [==============================] - 416s 701ms/step - loss: 0.4281 - acc: 0.9930 - val_loss: 0.2390 - val_acc: 0.9952
Epoch 3/100
593/593 [==============================] - 402s 678ms/step - loss: 0.1609 - acc: 0.9935 - val_loss: 0.1124 - val_acc: 0.9942
Epoch 4/100
593/593 [==============================] - 423s 712ms/step - loss: 0.0876 - acc: 0.9930 - val_loss: 0.0768 - val_acc: 0.9930
Epoch 5/100
593/593 [==============================] - 421s 709ms/step - loss: 0.0646 - acc: 0.9925 - val_loss: 0.0646 - val_acc: 0.9932
Epoch 6/100
593/593 [==============================] - 416s 701ms/step - loss: 0.0557 - acc: 0.9924 - val_loss: 0.0612 - val_acc: 0.9941
Epoch 7/100
593/593 [==============================] - 415s 701ms/step - loss: 0.0511 - acc: 0.9926 - val_loss: 0.0604 - val_acc: 0.9936
Epoch 8/100
593/593 [==============================] 

In [10]:
model_input = keras.Input(shape=(MAX_SENTENSE_LEN, ))
x = keras.layers.Embedding(MAX_TOKENS, EMBEDDING_SIZE, weights=[embedding_matrix])(model_input)
x = keras.layers.Bidirectional(keras.layers.GRU(64, return_sequences=True, activation="relu", recurrent_dropout=0.1))(x)
x = keras.layers.SpatialDropout1D(0.1)(x)
x = keras.layers.Bidirectional(keras.layers.GRU(32, return_sequences=True, activation="relu", recurrent_dropout=0.1))(x)
x = keras.layers.GlobalMaxPool1D()(x)
x = keras.layers.Dense(32, activation="relu")(x)
model_output = keras.layers.Dense(len(LABELS), activation="sigmoid")(x)

model = keras.Model(inputs=model_input, outputs=model_output)
model.compile(keras.optimizers.Adam(3e-4), loss='binary_crossentropy', metrics=['acc'])

hist = model.fit(x_train, 
                 y_train, 
                 batch_size=256,
                 shuffle=True,
                 epochs=100, 
                 validation_split=0.05,
                 callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss',
                                                          patience=1, 
                                                          verbose=1)], 
                 verbose=1)

y_pred = model.predict(x_test)
submission[LABELS] = y_pred
submission.to_csv('93.submission.csv', index=False)

# ~ 0.97006

Epoch 1/100
593/593 [==============================] - 453s 765ms/step - loss: 0.1224 - acc: 0.9089 - val_loss: 0.0530 - val_acc: 0.9895
Epoch 2/100
593/593 [==============================] - 448s 755ms/step - loss: 0.0485 - acc: 0.9918 - val_loss: 0.0493 - val_acc: 0.9861
Epoch 3/100
593/593 [==============================] - 447s 755ms/step - loss: 0.0442 - acc: 0.9894 - val_loss: 0.0480 - val_acc: 0.9841
Epoch 4/100
593/593 [==============================] - 448s 755ms/step - loss: 0.0412 - acc: 0.9831 - val_loss: 0.0475 - val_acc: 0.9714
Epoch 5/100
593/593 [==============================] - 447s 754ms/step - loss: 0.0383 - acc: 0.9714 - val_loss: 0.0478 - val_acc: 0.9816
Epoch 00005: early stopping


In [11]:
model_input = keras.Input(shape=(MAX_SENTENSE_LEN, ))
x = keras.layers.Embedding(MAX_TOKENS, EMBEDDING_SIZE, weights=[embedding_matrix])(model_input)
x = keras.layers.Bidirectional(keras.layers.GRU(64, return_sequences=True, activation="relu", recurrent_dropout=0.1))(x)
x = keras.layers.SpatialDropout1D(0.1)(x)
x = keras.layers.Bidirectional(keras.layers.GRU(32, return_sequences=True, activation="relu", recurrent_dropout=0.1))(x)
x = keras.layers.GlobalMaxPool1D()(x)
x = keras.layers.Dense(32, activation="relu")(x)
model_output = keras.layers.Dense(len(LABELS), activation="sigmoid")(x)

model = keras.Model(inputs=model_input, outputs=model_output)
model.compile(keras.optimizers.Adam(3e-4), loss='binary_crossentropy', metrics=['acc'])

hist = model.fit(x_train, 
                 y_train, 
                 batch_size=32,
                 shuffle=True,
                 epochs=100, 
                 validation_split=0.05,
                 callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss',
                                                          patience=1, 
                                                          verbose=1)], 
                 verbose=1)

y_pred = model.predict(x_test)
submission[LABELS] = y_pred
submission.to_csv('94.submission.csv', index=False)

# ~ 0.97672

Epoch 1/100
4738/4738 [==============================] - 1693s 357ms/step - loss: 0.0624 - acc: 0.9664 - val_loss: 0.0491 - val_acc: 0.9931
Epoch 2/100
4738/4738 [==============================] - 1653s 349ms/step - loss: 0.0424 - acc: 0.9897 - val_loss: 0.0444 - val_acc: 0.9950
Epoch 3/100
4738/4738 [==============================] - 1642s 347ms/step - loss: 0.0376 - acc: 0.9831 - val_loss: 0.0443 - val_acc: 0.9934
Epoch 4/100
4738/4738 [==============================] - 1652s 349ms/step - loss: 0.0335 - acc: 0.9359 - val_loss: 0.0463 - val_acc: 0.9919
Epoch 00004: early stopping


In [13]:
model_input = keras.Input(shape=(MAX_SENTENSE_LEN,))
x = keras.layers.Embedding(MAX_TOKENS, EMBEDDING_SIZE, weights=[embedding_matrix])(model_input)
x = keras.layers.Bidirectional(keras.layers.LSTM(50, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(x)
x = keras.layers.GlobalMaxPool1D()(x)
x = keras.layers.Dense(50, activation="relu")(x)
x = keras.layers.Dropout(0.1)(x)
x = keras.layers.Dense(6, activation="sigmoid")(x)
model = keras.Model(inputs=model_input, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

hist = model.fit(x_train, 
                 y_train, 
                 batch_size=256,
                 shuffle=True,
                 epochs=100, 
                 validation_split=0.05,
                 callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss',
                                                          patience=1, 
                                                          verbose=1)], 
                 verbose=1)

y_pred = model.predict(x_test)
submission[LABELS] = y_pred
submission.to_csv('95.submission.csv', index=False)

# ~ 0.97625

Epoch 1/100
593/593 [==============================] - 314s 529ms/step - loss: 0.0835 - accuracy: 0.8609 - val_loss: 0.0488 - val_accuracy: 0.9952
Epoch 2/100
593/593 [==============================] - 338s 569ms/step - loss: 0.0450 - accuracy: 0.9453 - val_loss: 0.0457 - val_accuracy: 0.9935
Epoch 3/100
593/593 [==============================] - 336s 567ms/step - loss: 0.0388 - accuracy: 0.9489 - val_loss: 0.0459 - val_accuracy: 0.9900
Epoch 00003: early stopping


In [14]:
model_input = keras.Input(shape=(MAX_SENTENSE_LEN,))
x = keras.layers.Embedding(MAX_TOKENS, EMBEDDING_SIZE, weights=[embedding_matrix])(model_input)
x = keras.layers.Bidirectional(keras.layers.LSTM(50, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(x)
x = keras.layers.GlobalMaxPool1D()(x)
x = keras.layers.Dense(50, activation="relu")(x)
x = keras.layers.Dropout(0.1)(x)
x = keras.layers.Dense(6, activation="sigmoid")(x)
model = keras.Model(inputs=model_input, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=32, epochs=2, validation_split=0.1);

y_pred = model.predict(x_test)
submission[LABELS] = y_pred
submission.to_csv('96.submission.csv', index=False)
# ~ 0.98040

Epoch 1/2
4488/4488 [==============================] - 1267s 282ms/step - loss: 0.0550 - accuracy: 0.9542 - val_loss: 0.0448 - val_accuracy: 0.9939
Epoch 2/2
4488/4488 [==============================] - 1247s 278ms/step - loss: 0.0386 - accuracy: 0.9681 - val_loss: 0.0447 - val_accuracy: 0.9905


# Parameter tuning

# Online augmentation

# Model selection / blending

# Post-processing

# Evaluation

# Reasoning

# Monitoring